# EEL891 - 2024.2 - Trabalho 2
### Regressão Multivariável - Estimar Preços de Imóveis
*Aluno:* Miguel Badany Cerne<br>*DRE:* 123370433<br>*Usuário Kaggle:* miguelbadanycerne<br>*ID Kaggle:* 20164449
<hr>

# 1. Baixar e Importar Bibliotecas

In [255]:
%pip install pandas
%pip install numpy
%pip install matplotlib
%pip install scikit-learn
%pip install seaborn

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


# 2. Exploração e Tratamento de Dados

In [256]:
df_treino = pd.read_csv('./conjunto_de_treinamento.csv')
df_treino.drop(['Id'], axis=1, inplace=True)
df_teste = pd.read_csv('./conjunto_de_teste.csv')
df_teste.drop(['Id'], axis=1, inplace=True)

In [257]:
df_treino.head(3).T

,0,1,2
tipo,Casa,Apartamento,Apartamento
bairro,Imbiribeira,Casa Amarela,Encruzilhada
tipo_vendedor,Imobiliaria,Imobiliaria,Imobiliaria
quartos,3,4,3
suites,3,4,1
vagas,5,2,0
area_util,223,157,53
area_extra,167,0,0
diferenciais,piscina e copa,piscina e churrasqueira,nenhum
churrasqueira,0,1,0


In [258]:
df_treino.columns

Index(['tipo', 'bairro', 'tipo_vendedor', 'quartos', 'suites', 'vagas',
       'area_util', 'area_extra', 'diferenciais', 'churrasqueira',
       'estacionamento', 'piscina', 'playground', 'quadra', 's_festas',
       's_jogos', 's_ginastica', 'sauna', 'vista_mar', 'preco'],
      dtype='object')

Notamos que as seguintes features não são numéricas:
- tipo
- bairro
- tipo_vendedor
- diferenciais<br>
Devemos tratar melhor cada uma dessas colunas posteriormente.

In [259]:
df_treino['diferenciais'].unique()

array(['piscina e copa', 'piscina e churrasqueira', 'nenhum',
       'playground', 'piscina e playground', 'sauna e playground',
       'piscina e sauna', 'piscina e quadra poliesportiva',
       'salao de festas', 'campo de futebol e playground',
       'piscina e estacionamento visitantes', 'copa e salao de festas',
       'esquina e salao de festas', 'piscina e salao de festas',
       'playground e sala de ginastica', 'copa e playground',
       'estacionamento visitantes e playground',
       'playground e salao de festas', 'churrasqueira e playground',
       'churrasqueira e copa', 'churrasqueira e salao de festas', 'copa',
       'esquina', 'piscina', 'playground e quadra poliesportiva',
       'estacionamento visitantes', 'piscina e esquina', 'copa e esquina',
       'estacionamento visitantes e salao de festas',
       'piscina e campo de futebol',
       'frente para o mar e salao de festas', 'churrasqueira',
       'churrasqueira e campo de futebol', 'piscina e frente para 

Para viabilizar o One-Hot Encoding para esta coluna, faremos um mapeamento dessas classes para diminuir o número total de classes.
1. Simplificamos as categorias, aglutinando todas combinações de mais de um elemento no nome do primeiro elemento, seguido de '++'
2. Anulamos os valores 'nenhum', 'churrasqueira', 'estacionamento'... haja vista que, no conjunto de dados, temos colunas exclusivas para estes valores

In [260]:
valores_diferenciais = set(df_treino['diferenciais'].unique().tolist() + df_teste['diferenciais'].unique().tolist())

diferenciais = {valor: valor for valor in valores_diferenciais}

for valor in valores_diferenciais:
    if 'futebol e' in valor:
        diferenciais[valor] = 'futebol++'
    if 'care e' in valor:
        diferenciais[valor] = 'children care++'
    if 'churrasqueira e' in valor:
        diferenciais[valor] = 'churrasqueira++'
    if 'copa e' in valor:
        diferenciais[valor] = 'copa++'
    if 'esquina e' in valor:
        diferenciais[valor] = 'esquina++'
    if 'estacionamento visitantes e' in valor:
        diferenciais[valor] = 'estacionamento visita++'
    if 'mar e' in valor:
        diferenciais[valor] = 'mar++'
    if 'piscina e' in valor:
        diferenciais[valor] = 'piscina++'
    if 'playground e' in valor:
        diferenciais[valor] = 'playground++'
    if 'quadra poliesportiva e' in valor:
        diferenciais[valor] = 'quadra++'
    if 'ginastica e' in valor:
        diferenciais[valor] = 'ginastica++'
    if 'festas e' in valor:
        diferenciais[valor] = 'festas++'
    if 'jogos e' in valor:
        diferenciais[valor] = 'festas++'
    if 'sauna e' in valor:
        diferenciais[valor] = 'sauna++'
    if 'hidromassagem e' in valor:
        diferenciais[valor] = 'hidromassagem++'
    if valor == 'churrasqueira':
        diferenciais[valor] = np.nan
    if valor == 'estacionamento':
        diferenciais[valor] = np.nan
    if valor == 'piscina':
        diferenciais[valor] = np.nan
    if valor == 'playground':
        diferenciais[valor] = np.nan
    if valor == 'quadra poliesportiva':
        diferenciais[valor] = np.nan
    if valor == 'salao de festas':
        diferenciais[valor] = np.nan
    if valor == 'salao de jogos':
        diferenciais[valor] = np.nan
    if valor == 'sauna':
        diferenciais[valor] = np.nan
    if valor == 'frente para o mar':
        diferenciais[valor] = np.nan
    if valor == 'nenhum':
        diferenciais[valor] = np.nan
    

diferenciais

{'frente para o mar e esquina': 'mar++',
 'campo de futebol e estacionamento visitantes': 'futebol++',
 'piscina e campo de futebol': 'piscina++',
 'piscina e playground': 'piscina++',
 'frente para o mar': nan,
 'campo de futebol e esquina': 'futebol++',
 'salao de festas e vestiario': 'festas++',
 'piscina e churrasqueira': 'piscina++',
 'piscina e children care': 'piscina++',
 'piscina e copa': 'piscina++',
 'hidromassagem e salao de festas': 'hidromassagem++',
 'frente para o mar e copa': 'mar++',
 'copa': 'copa',
 'esquina e salao de festas': 'esquina++',
 'estacionamento visitantes e salao de jogos': 'estacionamento visita++',
 'campo de futebol e copa': 'futebol++',
 'estacionamento visitantes e playground': 'estacionamento visita++',
 'campo de futebol e quadra poliesportiva': 'futebol++',
 'churrasqueira e copa': 'churrasqueira++',
 'playground e quadra poliesportiva': 'playground++',
 'churrasqueira e children care': 'churrasqueira++',
 'sauna e copa': 'sauna++',
 'playground

In [261]:
df_treino = df_treino.replace(diferenciais)
df_teste = df_teste.replace(diferenciais)

In [262]:
df_treino.head(10).T

,0,1,2,3,4,5,6,7,8,9
tipo,Casa,Apartamento,Apartamento,Apartamento,Apartamento,Apartamento,Apartamento,Apartamento,Apartamento,Apartamento
bairro,Imbiribeira,Casa Amarela,Encruzilhada,Boa Viagem,Rosarinho,Boa Vista,Rosarinho,Espinheiro,Boa Viagem,Boa Viagem
tipo_vendedor,Imobiliaria,Imobiliaria,Imobiliaria,Imobiliaria,Imobiliaria,Imobiliaria,Imobiliaria,Imobiliaria,Imobiliaria,Imobiliaria
quartos,3,4,3,4,2,3,4,3,4,3
suites,3,4,1,3,1,0,2,1,4,1
vagas,5,2,0,2,1,1,0,1,3,1
area_util,223,157,53,149,54,130,105,124,170,72
area_extra,167,0,0,0,0,0,0,0,0,0
diferenciais,piscina++,piscina++,NaN,piscina++,piscina++,NaN,piscina++,NaN,piscina++,NaN
churrasqueira,0,1,0,1,1,0,0,0,1,0


In [263]:
# Checando a coluna 'diferenciais'
df_treino['diferenciais'].unique()

array(['piscina++', nan, 'sauna++', 'futebol++', 'copa++', 'esquina++',
       'playground++', 'estacionamento visita++', 'churrasqueira++',
       'copa', 'esquina', 'estacionamento visitantes', 'mar++',
       'ginastica++', 'festas++', 'sala de ginastica', 'quadra++',
       'children care', 'vestiario', 'children care++'], dtype=object)

In [264]:
df_teste['diferenciais'].unique()

array(['copa++', nan, 'piscina++', 'estacionamento visita++',
       'playground++', 'copa', 'futebol++', 'mar++', 'churrasqueira++',
       'estacionamento visitantes', 'quadra++', 'sauna++', 'festas++',
       'ginastica++', 'hidromassagem++', 'vestiario', 'esquina++',
       'esquina', 'sala de ginastica'], dtype=object)

## 2.1. Checando as colunas categóricas

Com o auxílio do google, conseguimos classificar todos bairros em 3 categorias: baixa (classe baixa), media (classe média) e alta (classe alta). Assim, melhoramos a eficácia do One-Hot Encoding novamente.

In [265]:
bairros = {'Aflitos': 'baixa',
 'Afogados':'baixa',
 'Agua Fria':'baixa',
 'Apipucos':'media',
 'Areias':'media',
 'Arruda':'media',
 'Barro':'baixa',
 'Beberibe':'baixa',
 'Beira Rio':'baixa',
 'Benfica':'baixa',
 'Boa Viagem':'alta',
 'Boa Vista':'media',
 'Bongi':'media',
 'Cajueiro':'media',
 'Campo Grande':'media',
 'Casa Amarela':'media',
 'Casa Forte':'alta',
 'Caxanga':'media',
 'Centro':'media',
 'Cid Universitaria':'media',
 'Coelhos':'baixa',
 'Cohab':'baixa',
 'Cordeiro':'media',
 'Derby':'media',
 'Dois Irmaos':'media',
 'Encruzilhada':'media',
 'Engenho do Meio':'media',
 'Espinheiro':'alta',
 'Estancia':'baixa',
 'Fundao':'baixa',
 'Gracas':'alta',
 'Guabiraba':'baixa',
 'Hipodromo':'baixa',
 'Ibura': 'baixa',
 'Ilha do Leite':'media',
 'Ilha do Retiro':'alta',
 'Imbiribeira':'media',
 'Ipsep':'media',
 'Iputinga':'media',
 'Jaqueira':'alta',
 'Jd S Paulo':'media',
 'Lagoa do Araca':'media',
 'Macaxeira':'baixa',
 'Madalena':'alta',
 'Monteiro':'media',
 'Paissandu':'alta',
 'Parnamirim':'media',
 'Piedade':'baixa',
 'Pina':'media',
 'Poco':'baixa',
 'Poco da Panela':'media',
 'Ponto de Parada':'baixa',
 'Prado':'media',
 'Recife':'media',
 'Rosarinho':'media',
 'S Jose':'alta',
 'San Martin':'media',
 'Sancho':'media',
 'Santana':'media',
 'Setubal':'alta',
 'Soledade':'media',
 'Sto Amaro':'media',
 'Sto Antonio':'media',
 'Tamarineira':'alta',
 'Tejipio':'media',
 'Torre':'media',
 'Torreao':'media',
 'Varzea':'media',
 'Zumbi':'baixa'}

df_treino = df_treino.replace(bairros)
df_teste = df_teste.replace(bairros)

In [266]:
# Checando classificação dos bairros
df_treino['bairro'].unique()

array(['media', 'alta', 'baixa'], dtype=object)

In [267]:
print(df_treino['tipo'].unique())
print(df_treino['tipo_vendedor'].unique())

['Casa' 'Apartamento' 'Quitinete' 'Loft']
['Imobiliaria' 'Pessoa Fisica']


Estudando as duas últimas features categóricas notamos que elas não são problemáticas para a aplicação do One-Hot Encoding. Então, podemos finalizar o tratamento dessas features.

### 2.1.1. Aplicação do One-Hot

In [268]:
df_treino = pd.get_dummies(df_treino, columns=['tipo', 'bairro', 'tipo_vendedor', 'diferenciais'], dtype=int)
df_teste = pd.get_dummies(df_teste, columns=['tipo', 'bairro', 'tipo_vendedor', 'diferenciais'], dtype=int)

Agora devemos checar se há alguma(s) coluna(s) em somente um dos conjuntos de dados e, caso haja, removê-la(s)

In [269]:
colunas_treino = set(df_treino.columns.tolist())
colunas_teste = set(df_teste.columns.tolist() + ['preco'])

comuns = colunas_treino & colunas_teste

nao_em_treino = colunas_teste - colunas_treino
nao_em_teste = colunas_treino - colunas_teste

# Remoção das colunas:
if len(nao_em_teste) != 0:
    df_treino.drop(list(nao_em_teste), axis=1, inplace=True)
if len(nao_em_treino) != 0:
    df_teste.drop(list(nao_em_treino), axis=1, inplace=True)

## 2.2. Remoção dos Outliers

Como recomendado pelo professor, removemos todos imóveis, cujos valores não pertencem ao intervalo [50.000, 5.000.000]

In [270]:
df_treino = df_treino[(df_treino['preco'] > 50_000) & (df_treino['preco'] < 5_000_000)]

## 2.3. Coeficiente de Pearson

Agora, realizamos a análise do coeficiente de Pearson e removemos as features com coeficientes baixos.

In [271]:
from scipy.stats import pearsonr

att_categoricos = [ x for x in df_treino.columns if df_treino[x].dtype == 'object']

for v in att_categoricos:
    print(f'\n{v:15}: {len(df_treino[v].unique())} valores únicos')
    print(f'{v}: {df_treino[v].unique()}')

colunas = df_treino.columns
colunas_baixo_pearson = []
for col in colunas:
    pearson = pearsonr(df_treino[col], df_treino["preco"])[0]
    print(f'{col:10} = {pearson:.3f}')
    if abs(pearson) < 0.1:
        colunas_baixo_pearson.append(col)

quartos    = 0.591
suites     = 0.711
vagas      = 0.559
area_util  = 0.605
area_extra = 0.042
churrasqueira = 0.050
estacionamento = -0.046
piscina    = 0.084
playground = -0.050
quadra     = -0.008
s_festas   = -0.074
s_jogos    = -0.013
s_ginastica = 0.006
sauna      = 0.145
vista_mar  = 0.191
preco      = 1.000
tipo_Apartamento = -0.117
tipo_Casa  = 0.123
tipo_Loft  = -0.016
bairro_alta = 0.208
bairro_baixa = -0.012
bairro_media = -0.206
tipo_vendedor_Imobiliaria = 0.029
tipo_vendedor_Pessoa Fisica = -0.029
diferenciais_churrasqueira++ = 0.037
diferenciais_copa = 0.008
diferenciais_copa++ = 0.064
diferenciais_esquina = -0.026
diferenciais_esquina++ = -0.001
diferenciais_estacionamento visita++ = -0.029
diferenciais_estacionamento visitantes = -0.049
diferenciais_festas++ = -0.013
diferenciais_futebol++ = -0.015
diferenciais_ginastica++ = -0.003
diferenciais_mar++ = 0.120
diferenciais_piscina++ = 0.090
diferenciais_playground++ = -0.037
diferenciais_quadra++ = 0.015
diferenciais_sal

In [272]:
# Removendo as colunas com baixo pearson
df_treino.drop(colunas_baixo_pearson, axis=1, inplace=True)
df_teste.drop(colunas_baixo_pearson, axis=1, inplace=True)

# 3. Regressão

In [273]:
from sklearn.model_selection import train_test_split

X = df_treino.drop(['preco'], axis=1)
y = df_treino['preco']

X_treino, X_teste, y_treino, y_teste = train_test_split(
    X,
    y,
    test_size=1500,
    random_state=0
)

## 3.1. Configuração do Algoritmo

In [274]:
from sklearn.preprocessing import StandardScaler

escala = StandardScaler()
escala.fit(X_treino)

X_treino = escala.transform(X_treino)
X_teste = escala.transform(X_teste)

### 3.1.1. Tentativa 1: Regressão Linear

In [275]:
import math
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score


regressor_linear = LinearRegression()

regressor_linear.fit(X_treino, y_treino)

y_pred_treino = regressor_linear.predict(X_treino)
y_pred_teste = regressor_linear.predict(X_teste)

mse_in = mean_squared_error(y_treino, y_pred_treino)
rmse_in = math.sqrt(mse_in)
r2_in = r2_score(y_treino, y_pred_treino)
mse_out = mean_squared_error(y_teste, y_pred_teste)
rmse_out = math.sqrt(mse_out)
r2_out = r2_score(y_teste, y_pred_teste)

print(f'{"MSE":10} {mse_in:10.2f} {mse_out:10.2f}')
print(f'{"RMSE":10} {rmse_in:10.2f} {rmse_out:10.2f}')
print(f'{"R2":10} {r2_in:10.2f} {r2_out:10.2f}')

rmspe = (np.sqrt(np.mean(np.square((y_teste - y_pred_teste) / y_teste))))

print(f'{"RMSPE":10} {rmspe:10.2f}')

MSE        100022043770.60 127900164535.87
RMSE        316262.62  357631.32
R2               0.67       0.63
RMSPE            0.41


O resultado obteve resultados pouco precisos, sendo assim, subimos um grau de complexidade e usaremos um *regressor polinomial*

### 3.1.2. Tentativa 2: Regressão Polinomial

In [276]:
from sklearn.preprocessing import PolynomialFeatures


for k in range(1,4):
    pf = PolynomialFeatures(degree=k)
    pf.fit(X_treino)

    X_treino_poly = pf.transform(X_treino)
    X_teste_poly = pf.transform(X_teste)

    regressor_linear_poly = LinearRegression()

    regressor_linear_poly.fit(X_treino_poly, y_treino)

    y_pred_treino_poly = regressor_linear_poly.predict(X_treino_poly)
    y_pred_teste_poly = regressor_linear_poly.predict(X_teste_poly)

    mse_in_poly = mean_squared_error(y_treino, y_pred_treino_poly)
    rmse_in_poly = math.sqrt(mse_in_poly)
    r2_in_poly = r2_score(y_treino, y_pred_treino_poly)
    rmspe = (np.sqrt(np.mean(np.square((y_teste - y_pred_teste_poly) / y_teste))))
    mse_out_poly = mean_squared_error(y_teste, y_pred_teste_poly)
    rmse_out_poly = math.sqrt(mse_out_poly)
    r2_out_poly = r2_score(y_teste, y_pred_teste_poly)

    print(f'{k:4} {rmse_in_poly:10.2f} {rmse_out_poly:10.2f} {rmspe:10.2f}')

   1  316262.62  357631.32       0.41
   2  255630.22  317707.86       0.34
   3  217442.87  546329.04       0.56


In [277]:
df_treino_final = df_treino.sample(frac=1, random_state=0)
X_final = df_treino_final.drop(['preco'], axis=1).to_numpy()
y_final = df_treino_final['preco'].to_numpy()

X_teste_final = df_teste.to_numpy()

X_treino_final = X_final
y_treino_final = y_final

X_teste_final = X_teste_final

### 3.1.3. Tentativa 3: Regressor Random Forest

Ajustando os parâmetros

In [278]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold, cross_val_score


regressor = RandomForestRegressor(
    bootstrap=True,
    max_depth=None,
    min_samples_leaf=2,
    min_samples_split=5,
    n_estimators=200,
    random_state=0
)
regressor.fit(X_treino, y_treino)

y_pred_teste = regressor.predict(X_teste_final)
kfold = KFold(n_splits = 5, shuffle = True, random_state = 0)
resultado = cross_val_score(regressor, X_treino, y_treino, cv = kfold)
print(f'K-Fold (Rˆ2) Scores: {resultado}')
print(f'Média do Rˆ2 por validação cruzada K-Fold: {resultado.mean()}')

K-Fold (Rˆ2) Scores: [0.84819442 0.8026661  0.78520532 0.77571115 0.8178226 ]
Média do Rˆ2 por validação cruzada K-Fold: 0.8059199188885457


In [279]:
for i in range(1,10):
    regressor = RandomForestRegressor(
        n_estimators=i,
        random_state=0
    )
    kfold = KFold(n_splits = 5, shuffle = True, random_state = 0)
    resultado = cross_val_score(regressor, X_treino, y_treino, cv = kfold)
    print(f'Iteração {i}')
    print(f'K-Fold (Rˆ2) Scores: {resultado}')
    print(f'Média do Rˆ2 por validação cruzada K-Fold: {resultado.mean()}')

Iteração 1
K-Fold (Rˆ2) Scores: [0.70031131 0.70490294 0.77165093 0.57388816 0.71817124]
Média do Rˆ2 por validação cruzada K-Fold: 0.6937849163009562
Iteração 2
K-Fold (Rˆ2) Scores: [0.76728215 0.71376432 0.79143834 0.67714651 0.76799836]
Média do Rˆ2 por validação cruzada K-Fold: 0.7435259371438729
Iteração 3
K-Fold (Rˆ2) Scores: [0.7824233  0.76305524 0.78319028 0.70737337 0.80060566]
Média do Rˆ2 por validação cruzada K-Fold: 0.7673295675606027
Iteração 4
K-Fold (Rˆ2) Scores: [0.77804103 0.78208784 0.7790906  0.72038487 0.80635861]
Média do Rˆ2 por validação cruzada K-Fold: 0.7731925907824043
Iteração 5
K-Fold (Rˆ2) Scores: [0.78170928 0.7838926  0.77102901 0.73488157 0.81547698]
Média do Rˆ2 por validação cruzada K-Fold: 0.777397887131035
Iteração 6
K-Fold (Rˆ2) Scores: [0.79202788 0.78184506 0.76828242 0.75333923 0.81413245]
Média do Rˆ2 por validação cruzada K-Fold: 0.7819254081443812
Iteração 7
K-Fold (Rˆ2) Scores: [0.79732114 0.79341757 0.77005249 0.7558637  0.80955988]
Média 

Geração do CSV

In [280]:
regressor = RandomForestRegressor(
    bootstrap=True,
    max_depth=None,
    min_samples_leaf=2,
    min_samples_split=10,
    n_estimators=300,
    random_state=0
)
regressor.fit(X_treino_final, y_treino_final)
y_pred_teste = regressor.predict(X_teste_final)

df_teste_original = pd.read_csv('./conjunto_de_teste.csv')
df_teste_original['preco'] = y_pred_teste

resposta_final = pd.DataFrame({'Id': df_teste_original.pop('Id'), 'preco': np.squeeze((y_pred_teste))})
resposta_final.to_csv('answer.csv', index=False)